# Task
Predict "Customer Lifetime Value" using the `AutoInsurance.csv` dataset, building and evaluating Linear Regression, Random Forest Regressor, and Gradient Boosting Regressor models. Select the best performing model and deploy it with a Gradio interface.

## Load and Initial Data Inspection

### Subtask:
Load the 'AutoInsurance.csv' dataset and perform an initial inspection to understand its structure, identify data types, and check for missing values.


**Reasoning**:
To address all instructions in the subtask, I will first import the pandas library, then load the specified CSV file into a DataFrame, display its head, print a concise summary, and finally show the count of missing values for each column.



In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/AutoInsurance.csv')
print("Dataset loaded successfully. Displaying the first 5 rows:")
# Display the first 5 rows
print(df.head())

print("\nConcise summary of the DataFrame:")
# Print the concise summary
print(df.info())

print("\nNumber of missing values per column:")
# Print the number of missing values for each column
print(df.isnull().sum())

Dataset loaded successfully. Displaying the first 5 rows:
  Customer       State  Customer Lifetime Value Response  Coverage Education  \
0  BU79786  Washington              2763.519279       No     Basic  Bachelor   
1  QZ44356     Arizona              6979.535903       No  Extended  Bachelor   
2  AI49188      Nevada             12887.431650       No   Premium  Bachelor   
3  WW63253  California              7645.861827       No     Basic  Bachelor   
4  HB64268  Washington              2813.692575       No     Basic  Bachelor   

  Effective To Date EmploymentStatus Gender  Income  ...  \
0           2/24/11         Employed      F   56274  ...   
1           1/31/11       Unemployed      F       0  ...   
2           2/19/11         Employed      F   48767  ...   
3           1/20/11       Unemployed      M       0  ...   
4          3/2/2011         Employed      M   43836  ...   

  Months Since Policy Inception Number of Open Complaints  Number of Policies  \
0                  

## Data Preprocessing and Feature Engineering

### Subtask:
Clean the data by handling missing values (if any), encoding categorical features (e.g., one-hot encoding), scaling numerical features, and preparing the target variable (Customer Lifetime Value) and features for model training. This step will also involve identifying and potentially dropping irrelevant columns.


**Reasoning**:
First, I will drop the 'Customer' and 'Effective To Date' columns as they are identified as irrelevant for model training, and then separate the target variable 'Customer Lifetime Value' from the features into 'y' and 'X' respectively, as instructed.



In [2]:
print("Original DataFrame shape:", df.shape)
# Drop irrelevant columns
df = df.drop(['Customer', 'Effective To Date'], axis=1)
print("DataFrame shape after dropping irrelevant columns:", df.shape)

# Separate target variable (y) and features (X)
y = df['Customer Lifetime Value']
X = df.drop('Customer Lifetime Value', axis=1)

print("\nShape of X (features):")
print(X.shape)
print("Shape of y (target):")
print(y.shape)
print("First 5 rows of y:")
print(y.head())

Original DataFrame shape: (9134, 24)
DataFrame shape after dropping irrelevant columns: (9134, 22)

Shape of X (features):
(9134, 21)
Shape of y (target):
(9134,)
First 5 rows of y:
0     2763.519279
1     6979.535903
2    12887.431650
3     7645.861827
4     2813.692575
Name: Customer Lifetime Value, dtype: float64


**Reasoning**:
Now that the irrelevant columns have been dropped and the target variable separated, I will identify the categorical and numerical features within the feature DataFrame `X` as per the next instruction. This involves iterating through the columns of `X` and classifying them based on their data types.



In [3]:
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print("Categorical Features:", categorical_features)
print("Number of Categorical Features:", len(categorical_features))
print("\nNumerical Features:", numerical_features)
print("Number of Numerical Features:", len(numerical_features))

Categorical Features: ['State', 'Response', 'Coverage', 'Education', 'EmploymentStatus', 'Gender', 'Location Code', 'Marital Status', 'Policy Type', 'Policy', 'Renew Offer Type', 'Sales Channel', 'Vehicle Class', 'Vehicle Size']
Number of Categorical Features: 14

Numerical Features: ['Income', 'Monthly Premium Auto', 'Months Since Last Claim', 'Months Since Policy Inception', 'Number of Open Complaints', 'Number of Policies', 'Total Claim Amount']
Number of Numerical Features: 7


**Reasoning**:
Following the identification of categorical and numerical features, I will now apply one-hot encoding to the categorical features using `pd.get_dummies()` with `drop_first=True` to prevent multicollinearity, as instructed. Then, I will replace the original categorical columns in `X` with these newly encoded columns.



In [4]:
print("Shape of X before one-hot encoding:", X.shape)

# Apply one-hot encoding to categorical features
X_categorical_encoded = pd.get_dummies(X[categorical_features], drop_first=True)

# Drop original categorical columns from X and concatenate with encoded features
X = X.drop(columns=categorical_features)
X = pd.concat([X, X_categorical_encoded], axis=1)

print("Shape of X after one-hot encoding:", X.shape)
print("First 5 rows of X after one-hot encoding:")
print(X.head())

Shape of X before one-hot encoding: (9134, 21)
Shape of X after one-hot encoding: (9134, 50)
First 5 rows of X after one-hot encoding:
   Income  Monthly Premium Auto  Months Since Last Claim  \
0   56274                    69                       32   
1       0                    94                       13   
2   48767                   108                       18   
3       0                   106                       18   
4   43836                    73                       12   

   Months Since Policy Inception  Number of Open Complaints  \
0                              5                          0   
1                             42                          0   
2                             38                          0   
3                             65                          0   
4                             44                          0   

   Number of Policies  Total Claim Amount  State_California  State_Nevada  \
0                   1          384.811147       

**Reasoning**:
Now that categorical features are encoded, the next step is to scale the numerical features as per the instructions. I will use `StandardScaler` to fit and transform the numerical columns in `X`, and then update `X` with these scaled values.



In [5]:
from sklearn.preprocessing import StandardScaler

print("Shape of X before scaling numerical features:", X.shape)

# Instantiate StandardScaler
scaler = StandardScaler()

# Apply StandardScaler to numerical features
X[numerical_features] = scaler.fit_transform(X[numerical_features])

print("Shape of X after scaling numerical features:", X.shape)
print("First 5 rows of X after scaling numerical features:")
print(X.head())


Shape of X before scaling numerical features: (9134, 50)
Shape of X after scaling numerical features: (9134, 50)
First 5 rows of X after scaling numerical features:
     Income  Monthly Premium Auto  Months Since Last Claim  \
0  0.612827             -0.703925                 1.678099   
1 -1.239617              0.022691                -0.208186   
2  0.365710              0.429596                 0.288205   
3 -1.239617              0.371467                 0.288205   
4  0.203390             -0.587666                -0.307465   

   Months Since Policy Inception  Number of Open Complaints  \
0                      -1.543287                   -0.42225   
1                      -0.217334                   -0.42225   
2                      -0.360680                   -0.42225   
3                       0.606907                   -0.42225   
4                      -0.145661                   -0.42225   

   Number of Policies  Total Claim Amount  State_California  State_Nevada  \
0     

## Split Data into Training and Testing Sets

### Subtask:
Divide the preprocessed dataset into training and testing sets to evaluate model performance on unseen data.


**Reasoning**:
To divide the preprocessed dataset into training and testing sets as instructed, I will first import the `train_test_split` function from `sklearn.model_selection` and then apply it to the `X` and `y` DataFrames with a `test_size` of 0.2 and a `random_state` of 42.



In [6]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (7307, 50)
Shape of X_test: (1827, 50)
Shape of y_train: (7307,)
Shape of y_test: (1827,)


## Train and Evaluate Linear Regression Model

### Subtask:
Train a Linear Regression model on the training data. Then, evaluate its performance by calculating and printing the R-squared scores for both the training and testing sets. Analyze these scores to check for overfitting.


**Reasoning**:
To train and evaluate the Linear Regression model, I will import the necessary classes, instantiate the model, fit it to the training data, make predictions, and then calculate and print the R-squared scores for both training and testing sets as instructed.



In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Instantiate Linear Regression model
linear_model = LinearRegression()

# Fit the model to the training data
linear_model.fit(X_train, y_train)

# Make predictions on training and testing sets
y_pred_train_linear = linear_model.predict(X_train)
y_pred_test_linear = linear_model.predict(X_test)

# Calculate R-squared for training set
r2_train_linear = r2_score(y_train, y_pred_train_linear)
print(f"Linear Regression Training R-squared: {r2_train_linear:.4f}")

# Calculate R-squared for testing set
r2_test_linear = r2_score(y_test, y_pred_test_linear)
print(f"Linear Regression Testing R-squared: {r2_test_linear:.4f}")

Linear Regression Training R-squared: 0.1714
Linear Regression Testing R-squared: 0.1551


The R-squared score for the training set is 0.1714, and for the testing set, it is 0.1551. These scores are quite low, indicating that the Linear Regression model does not explain a large proportion of the variance in 'Customer Lifetime Value'. The scores for training and testing are close, suggesting that the model is not overfitting (i.e., it's not performing significantly better on the training data than on unseen data). However, the overall low R-squared values suggest that the model is underfitting, meaning it is too simple to capture the underlying patterns in the data effectively.

## Train and Evaluate Random Forest Regressor Model

### Subtask:
Train a Random Forest Regressor model on the training data. Then, evaluate its performance by calculating and printing the R-squared scores for both the training and testing sets. Analyze these scores to check for overfitting.

**Reasoning**:
Following the introduction of the subtask for Random Forest Regressor, I will now import the necessary libraries, instantiate the Random Forest Regressor model, train it on the `X_train` and `y_train` data, and then make predictions on both the training and testing sets to prepare for R-squared score calculation.



In [8]:
from sklearn.ensemble import RandomForestRegressor

# Instantiate Random Forest Regressor model
rf_model = RandomForestRegressor(random_state=42)

# Fit the model to the training data
rf_model.fit(X_train, y_train)

# Make predictions on training and testing sets
y_pred_train_rf = rf_model.predict(X_train)
y_pred_test_rf = rf_model.predict(X_test)

print("Random Forest Regressor model trained and predictions made.")

Random Forest Regressor model trained and predictions made.


**Reasoning**:
Now that the Random Forest Regressor model has been trained and predictions have been made, I will calculate and print the R-squared scores for both the training and testing sets to evaluate its performance, as instructed in the subtask.



In [9]:
from sklearn.metrics import r2_score

# Calculate R-squared for training set
r2_train_rf = r2_score(y_train, y_pred_train_rf)
print(f"Random Forest Regressor Training R-squared: {r2_train_rf:.4f}")

# Calculate R-squared for testing set
r2_test_rf = r2_score(y_test, y_pred_test_rf)
print(f"Random Forest Regressor Testing R-squared: {r2_test_rf:.4f}")

Random Forest Regressor Training R-squared: 0.9596
Random Forest Regressor Testing R-squared: 0.6901


The Random Forest Regressor achieved a training R-squared score of 0.9596 and a testing R-squared score of 0.6901. The high R-squared on the training set (0.9596) suggests that the model fits the training data very well. However, there is a significant drop in performance on the testing set (0.6901).

This discrepancy indicates that the Random Forest Regressor is likely overfitting the training data. While it captures complex patterns in the training data, it does not generalize as effectively to unseen data. This suggests that the model might be too complex for the given dataset or that hyperparameter tuning is needed to reduce overfitting and improve its generalization capabilities.

## Train and Evaluate Gradient Boosting Regressor Model

### Subtask:
Train a Gradient Boosting Regressor model on the training data. Then, evaluate its performance by calculating and printing the R-squared scores for both the training and testing sets. Analyze these scores to check for overfitting.

**Reasoning**:
Following the introduction of the subtask for the Gradient Boosting Regressor, I will now import the necessary libraries, instantiate the Gradient Boosting Regressor model, train it on the `X_train` and `y_train` data, and then make predictions on both the training and testing sets.



In [10]:
from sklearn.ensemble import GradientBoostingRegressor

# Instantiate Gradient Boosting Regressor model
gbr_model = GradientBoostingRegressor(random_state=42)

# Fit the model to the training data
gbr_model.fit(X_train, y_train)

# Make predictions on training and testing sets
y_pred_train_gbr = gbr_model.predict(X_train)
y_pred_test_gbr = gbr_model.predict(X_test)

print("Gradient Boosting Regressor model trained and predictions made.")

Gradient Boosting Regressor model trained and predictions made.


**Reasoning**:
Now that the Gradient Boosting Regressor model has been trained and predictions have been made, I will calculate and print the R-squared scores for both the training and testing sets to evaluate its performance, as instructed in the subtask.



In [11]:
from sklearn.metrics import r2_score

# Calculate R-squared for training set
r2_train_gbr = r2_score(y_train, y_pred_train_gbr)
print(f"Gradient Boosting Regressor Training R-squared: {r2_train_gbr:.4f}")

# Calculate R-squared for testing set
r2_test_gbr = r2_score(y_test, y_pred_test_gbr)
print(f"Gradient Boosting Regressor Testing R-squared: {r2_test_gbr:.4f}")

Gradient Boosting Regressor Training R-squared: 0.7349
Gradient Boosting Regressor Testing R-squared: 0.6662


The Gradient Boosting Regressor achieved a training R-squared score of 0.7349 and a testing R-squared score of 0.6662. Both scores are significantly higher than those from the Linear Regression model, indicating a much better fit to the data. Compared to the Random Forest Regressor, the training R-squared is lower (0.7349 vs 0.9596), but the testing R-squared is somewhat comparable (0.6662 vs 0.6901). The difference between the training and testing R-squared for the Gradient Boosting Regressor is smaller than that of the Random Forest Regressor, suggesting less overfitting. While there is still a drop in performance from training to testing, it's less pronounced, indicating better generalization than the Random Forest model without extensive hyperparameter tuning.

## Compare Models and Select the Best

### Subtask:
Compare the R-squared scores of the Linear Regression, Random Forest Regressor, and Gradient Boosting Regressor models on both the training and testing sets. Select the best performing model based on its generalization capabilities (i.e., testing R-squared score) and note the chosen model.

### Model Comparison and Selection

Let's summarize the R-squared scores for all models:

*   **Linear Regression**
    *   Training R-squared: 0.1714
    *   Testing R-squared: 0.1551
    *   *Analysis*: Very low R-squared, indicating underfitting. No significant overfitting.

*   **Random Forest Regressor**
    *   Training R-squared: 0.9596
    *   Testing R-squared: 0.6901
    *   *Analysis*: High training R-squared but a considerable drop on the testing set suggests overfitting. However, it achieved the highest testing R-squared.

*   **Gradient Boosting Regressor**
    *   Training R-squared: 0.7349
    *   Testing R-squared: 0.6662
    *   *Analysis*: Good training R-squared with a smaller drop on the testing set compared to Random Forest, indicating better generalization and less overfitting. Its testing R-squared is slightly lower than Random Forest.

**Conclusion:**

Based on the testing R-squared scores, the **Random Forest Regressor** model performs the best with a testing R-squared of **0.6901**. Although it shows signs of overfitting (large difference between training and testing scores), it generalizes better to unseen data compared to the other models, as evidenced by its superior testing R-squared. The Gradient Boosting Regressor is a close second with 0.6662, showing better generalization characteristics (less overfitting) but slightly lower predictive power on the test set for this specific run. For this task, we prioritize the highest predictive performance on unseen data (testing R-squared).

## Deploy Best Model with Gradio

### Subtask:
Create a user-friendly web interface using Gradio to deploy the selected best model, allowing users to input new data and get predictions for Customer Lifetime Value.


**Reasoning**:
First, I will identify the best performing model based on the R-squared scores from the previous steps, which is the Random Forest Regressor. Then, I will import the `gradio` library and define a `predict_clv` function that takes all relevant features as input, preprocesses them (one-hot encoding categorical features, scaling numerical features), and uses the selected `rf_model` to make a prediction. Finally, I will create a Gradio interface with appropriate input components for each feature and launch it, as instructed by the subtask.



In [12]:
import gradio as gr
import pandas as pd

# Best performing model is Random Forest Regressor (rf_model) based on testing R-squared
# (r2_test_rf: 0.6901, r2_test_gbr: 0.6662, r2_test_linear: 0.1551)

# Get unique values for dropdowns from the 'df' (which is the one after dropping irrelevant columns but before one-hot encoding)
categorical_unique_values = {}
for col in categorical_features:
    categorical_unique_values[col] = df[col].unique().tolist()

def predict_clv(*args):
    # The order of args must match the order of feature names defined for the Gradio interface
    # numerical_features: 'Income', 'Monthly Premium Auto', 'Months Since Last Claim', 'Months Since Policy Inception', 'Number of Open Complaints', 'Number of Policies', 'Total Claim Amount'
    # categorical_features: 'State', 'Response', 'Coverage', 'Education', 'EmploymentStatus', 'Gender', 'Location Code', 'Marital Status', 'Policy Type', 'Policy', 'Renew Offer Type', 'Sales Channel', 'Vehicle Class', 'Vehicle Size'

    input_data = {}
    arg_idx = 0
    # Populate numerical features
    for feature in numerical_features:
        input_data[feature] = [args[arg_idx]]
        arg_idx += 1
    # Populate categorical features
    for feature in categorical_features:
        input_data[feature] = [args[arg_idx]]
        arg_idx += 1

    input_df = pd.DataFrame(input_data)

    # Apply one-hot encoding to categorical features
    input_categorical_encoded = pd.get_dummies(input_df[categorical_features], drop_first=True)

    # Prepare the input_df for scaling and prediction
    # Drop original categorical columns from the input_df
    input_df_processed = input_df.drop(columns=categorical_features)
    # Concatenate with encoded features
    input_df_processed = pd.concat([input_df_processed, input_categorical_encoded], axis=1)

    # Reindex to ensure all columns from training (X.columns) are present and in the correct order
    # Fill missing columns (for categories not present in the single input row) with 0
    input_df_processed = input_df_processed.reindex(columns=X.columns, fill_value=0)

    # Scale numerical features using the pre-fitted scaler
    # Ensure 'Income' column exists in input_df_processed before scaling
    if 'Income' in input_df_processed.columns:
        input_df_processed[numerical_features] = scaler.transform(input_df_processed[numerical_features])
    else:
        # Handle case where numerical_features might not be directly in the reindexed DataFrame if X.columns didn't include them all
        # This scenario is unlikely given how X was created, but added for robustness
        for num_feat in numerical_features:
            if num_feat in input_df_processed.columns:
                input_df_processed[num_feat] = scaler.transform(input_df_processed[[num_feat]])

    # Make prediction using the best model (rf_model)
    prediction = rf_model.predict(input_df_processed)[0] # [0] to get the single scalar value

    return f"Predicted Customer Lifetime Value: ${prediction:.2f}"

# Create Gradio interface inputs
inputs = []
for feature in numerical_features:
    inputs.append(gr.Number(label=feature))

for feature in categorical_features:
    inputs.append(gr.Dropdown(choices=categorical_unique_values[feature], label=feature))

output = gr.Textbox(label="Predicted Customer Lifetime Value")

# Launch the Gradio interface
gr.Interface(
    fn=predict_clv,
    inputs=inputs,
    outputs=output,
    title="Customer Lifetime Value Prediction",
    description="Enter customer details to predict their Customer Lifetime Value."
).launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f96414a025478a3c25.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Final Task

### Subtask:
Summarize the entire process, including the data preprocessing steps, the performance of each model, the rationale for selecting the best model, and a demonstration of the deployed Gradio interface for predicting Customer Lifetime Value.


## Summary:

### Data Analysis Key Findings

*   **Initial Data Inspection**: The `AutoInsurance.csv` dataset, comprising 9134 entries across 24 columns, was loaded successfully. It was found to be clean, with no missing values in any column. Data types included 2 float, 6 integer, and 16 object (categorical) columns.
*   **Data Preprocessing**:
    *   Irrelevant columns 'Customer' and 'Effective To Date' were dropped.
    *   The target variable, 'Customer Lifetime Value', was separated.
    *   14 categorical features were identified and one-hot encoded (using `drop_first=True`), expanding the feature set from 21 to 50 columns.
    *   7 numerical features were identified and scaled using `StandardScaler`.
*   **Data Splitting**: The preprocessed data was split into training (7307 samples) and testing (1827 samples) sets with an 80/20 ratio.
*   **Model Performance Comparison**:
    *   **Linear Regression**: Showed very low R-squared scores (Training: 0.1714, Testing: 0.1551), indicating severe underfitting.
    *   **Random Forest Regressor**: Achieved a high training R-squared (0.9596) but a significantly lower testing R-squared (0.6901), suggesting overfitting. However, it delivered the highest predictive performance on unseen data.
    *   **Gradient Boosting Regressor**: Performed better than Linear Regression (Training R-squared: 0.7349, Testing R-squared: 0.6662). It exhibited less overfitting than the Random Forest, but its testing R-squared was slightly lower.
*   **Best Model Selection**: The **Random Forest Regressor** was selected as the best model for deployment due to its highest testing R-squared score of **0.6901**, demonstrating the best predictive power on unseen data among the evaluated models, despite showing signs of overfitting.
*   **Model Deployment**: The selected Random Forest Regressor model was successfully deployed using a Gradio web interface. The interface allows users to input numerical and categorical customer details, which are preprocessed using the same steps (one-hot encoding, scaling) as during training, to predict Customer Lifetime Value.

### Insights or Next Steps

*   **Overfitting Mitigation**: While the Random Forest Regressor had the best testing performance, the notable difference between its training (0.9596) and testing (0.6901) R-squared scores indicates overfitting. Further hyperparameter tuning (e.g., using GridSearchCV or RandomizedSearchCV) or techniques like pruning could be explored to improve its generalization capabilities and potentially reduce this gap without sacrificing too much predictive power.
*   **Feature Engineering Exploration**: Given the relatively low R-squared values across all models (even the best at 0.6901), exploring additional feature engineering techniques, such as creating interaction terms or polynomial features, could enhance the models' ability to capture more complex relationships in the data and improve predictive accuracy.


# Task
Improve the prediction of "Customer Lifetime Value" using the `AutoInsurance.csv` dataset by identifying and handling outliers in numerical features, retraining and evaluating Linear Regression, Random Forest Regressor, and Gradient Boosting Regressor models on the outlier-adjusted data, selecting the best performing model, and deploying it with a Gradio interface. Finally, summarize the impact of outlier removal on model performance.

## Identify and Remove Outliers

### Subtask:
Identify outliers in the numerical features of the dataset using an appropriate statistical method (e.g., IQR) and then remove or cap them to create a cleaned dataset. This step will use the `X` and `y` from the previous preprocessing steps.


**Reasoning**:
To identify and remove outliers using the IQR method, I will first initialize an empty boolean mask. Then, I will iterate through each numerical feature, calculate its Q1, Q3, and IQR, define the outlier bounds, and update the mask to mark outliers. Finally, I will use this mask to filter both `X` and `y` to create `X_cleaned` and `y_cleaned`, and print their new shapes.



In [13]:
import numpy as np

# 1. Initialize an empty boolean series `outlier_mask`
outlier_mask = pd.Series([False] * len(X), index=X.index)

# 2. Iterate through each numerical feature
for feature in numerical_features:
    # 3. Calculate the first quartile (Q1) and third quartile (Q3)
    Q1 = X[feature].quantile(0.25)
    Q3 = X[feature].quantile(0.75)

    # 4. Calculate the Interquartile Range (IQR)
    IQR = Q3 - Q1

    # 5. Determine the lower bound and upper bound for outlier detection
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # 6. Update the `outlier_mask` to mark `True` for any row where the feature's value is an outlier
    outlier_mask = outlier_mask | ((X[feature] < lower_bound) | (X[feature] > upper_bound))

# 7. Create a new DataFrame `X_cleaned` by filtering `X` using the inverse of the `outlier_mask`
X_cleaned = X[~outlier_mask]

# 8. Create a new Series `y_cleaned` by filtering `y` using the inverse of the `outlier_mask`
y_cleaned = y[~outlier_mask]

# 9. Print the shape of `X_cleaned` and `y_cleaned`
print(f"Original shape of X: {X.shape}")
print(f"Shape of X_cleaned after outlier removal: {X_cleaned.shape}")
print(f"Original shape of y: {y.shape}")
print(f"Shape of y_cleaned after outlier removal: {y_cleaned.shape}")

Original shape of X: (9134, 50)
Shape of X_cleaned after outlier removal: (6436, 50)
Original shape of y: (9134,)
Shape of y_cleaned after outlier removal: (6436,)


## Re-split Data into Training and Testing Sets (Outlier-Adjusted)

### Subtask:
Divide the outlier-adjusted dataset (X_cleaned, y_cleaned) into new training and testing sets to prepare for re-training the models.


**Reasoning**:
To divide the outlier-adjusted dataset into training and testing sets as instructed, I will use the `train_test_split` function from `sklearn.model_selection` with a `test_size` of 0.2 and a `random_state` of 42, then print the shapes of the resulting datasets.



In [14]:
from sklearn.model_selection import train_test_split

# Split the cleaned data into training and testing sets
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

print("Shape of X_train_cleaned:", X_train_cleaned.shape)
print("Shape of X_test_cleaned:", X_test_cleaned.shape)
print("Shape of y_train_cleaned:", y_train_cleaned.shape)
print("Shape of y_test_cleaned:", y_test_cleaned.shape)

Shape of X_train_cleaned: (5148, 50)
Shape of X_test_cleaned: (1288, 50)
Shape of y_train_cleaned: (5148,)
Shape of y_test_cleaned: (1288,)


## Retrain and Evaluate Linear Regression Model (Outlier-Adjusted)

### Subtask:
Retrain the Linear Regression model on the outlier-adjusted training data and evaluate its performance using R-squared scores for both training and testing sets. Analyze for overfitting/underfitting.


**Reasoning**:
To retrain and evaluate the Linear Regression model on the outlier-adjusted data, I will import necessary classes, instantiate the model, fit it to the cleaned training data, make predictions on both cleaned training and testing sets, and then calculate and print the R-squared scores for both.



In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Instantiate Linear Regression model
linear_model_cleaned = LinearRegression()

# Fit the model to the cleaned training data
linear_model_cleaned.fit(X_train_cleaned, y_train_cleaned)

# Make predictions on cleaned training and testing sets
y_pred_train_linear_cleaned = linear_model_cleaned.predict(X_train_cleaned)
y_pred_test_linear_cleaned = linear_model_cleaned.predict(X_test_cleaned)

# Calculate R-squared for cleaned training set
r2_train_linear_cleaned = r2_score(y_train_cleaned, y_pred_train_linear_cleaned)
print(f"Linear Regression (Outlier-Adjusted) Training R-squared: {r2_train_linear_cleaned:.4f}")

# Calculate R-squared for cleaned testing set
r2_test_linear_cleaned = r2_score(y_test_cleaned, y_pred_test_linear_cleaned)
print(f"Linear Regression (Outlier-Adjusted) Testing R-squared: {r2_test_linear_cleaned:.4f}")

Linear Regression (Outlier-Adjusted) Training R-squared: 0.1110
Linear Regression (Outlier-Adjusted) Testing R-squared: 0.0995


The Linear Regression model, after outlier removal, shows a training R-squared of 0.1110 and a testing R-squared of 0.0995. These scores are extremely low, even lower than the scores obtained before outlier removal (Training R-squared: 0.1714, Testing R-squared: 0.1551). This indicates that the Linear Regression model is still severely underfitting the data, meaning it is too simple to capture the underlying patterns, and the removal of outliers did not improve its performance; in fact, it slightly worsened it for this model. The very small difference between training and testing R-squared suggests no overfitting, but rather a consistent lack of fit.

**Reasoning**:
To retrain the Random Forest Regressor model on the outlier-adjusted data, I will import the necessary class, instantiate the model, fit it to the cleaned training data, and then make predictions on both the cleaned training and testing sets.



In [16]:
from sklearn.ensemble import RandomForestRegressor

# Instantiate Random Forest Regressor model with the same random_state
rf_model_cleaned = RandomForestRegressor(random_state=42)

# Fit the model to the cleaned training data
rf_model_cleaned.fit(X_train_cleaned, y_train_cleaned)

# Make predictions on cleaned training and testing sets
y_pred_train_rf_cleaned = rf_model_cleaned.predict(X_train_cleaned)
y_pred_test_rf_cleaned = rf_model_cleaned.predict(X_test_cleaned)

print("Random Forest Regressor (Outlier-Adjusted) model trained and predictions made.")

Random Forest Regressor (Outlier-Adjusted) model trained and predictions made.


**Reasoning**:
Now that the Random Forest Regressor model has been retrained on the outlier-adjusted data and predictions have been made, I will calculate and print the R-squared scores for both the cleaned training and testing sets to evaluate its performance, as instructed by the overarching task structure.



In [17]:
from sklearn.metrics import r2_score

# Calculate R-squared for cleaned training set
r2_train_rf_cleaned = r2_score(y_train_cleaned, y_pred_train_rf_cleaned)
print(f"Random Forest Regressor (Outlier-Adjusted) Training R-squared: {r2_train_rf_cleaned:.4f}")

# Calculate R-squared for cleaned testing set
r2_test_rf_cleaned = r2_score(y_test_cleaned, y_pred_test_rf_cleaned)
print(f"Random Forest Regressor (Outlier-Adjusted) Testing R-squared: {r2_test_rf_cleaned:.4f}")

Random Forest Regressor (Outlier-Adjusted) Training R-squared: 0.9565
Random Forest Regressor (Outlier-Adjusted) Testing R-squared: 0.7011


The Random Forest Regressor model, after outlier removal, shows a training R-squared of 0.9565 and a testing R-squared of 0.7011.

Compared to its performance before outlier removal (Training R-squared: 0.9596, Testing R-squared: 0.6901):
*   **Training R-squared** slightly decreased (from 0.9596 to 0.9565), indicating a minimal change in its fit to the training data. The model still fits the training data very well.
*   **Testing R-squared** slightly increased (from 0.6901 to 0.7011), suggesting a minor improvement in its generalization to unseen data after outliers were removed.

While the model still shows a notable difference between its training and testing R-squared, indicating some level of overfitting, the improvement in the testing R-squared after outlier removal is a positive sign. This suggests that removing outliers helped the model generalize slightly better, even if the primary issue of overfitting remains a consideration for further hyperparameter tuning.

The Random Forest Regressor model, after outlier removal, shows a training R-squared of 0.9565 and a testing R-squared of 0.7011.

Compared to its performance before outlier removal (Training R-squared: 0.9596, Testing R-squared: 0.6901):
*   **Training R-squared** slightly decreased (from 0.9596 to 0.9565), indicating a minimal change in its fit to the training data. The model still fits the training data very well.
*   **Testing R-squared** slightly increased (from 0.6901 to 0.7011), suggesting a minor improvement in its generalization to unseen data after outliers were removed.

While the model still shows a notable difference between its training and testing R-squared, indicating some level of overfitting, the improvement in the testing R-squared after outlier removal is a positive sign. This suggests that removing outliers helped the model generalize slightly better, even if the primary issue of overfitting remains a consideration for further hyperparameter tuning.

The Random Forest Regressor model, after outlier removal, shows a training R-squared of 0.9565 and a testing R-squared of 0.7011.

Compared to its performance before outlier removal (Training R-squared: 0.9596, Testing R-squared: 0.6901):
*   **Training R-squared** slightly decreased (from 0.9596 to 0.9565), indicating a minimal change in its fit to the training data. The model still fits the training data very well.
*   **Testing R-squared** slightly increased (from 0.6901 to 0.7011), suggesting a minor improvement in its generalization to unseen data after outliers were removed.

While the model still shows a notable difference between its training and testing R-squared, indicating some level of overfitting, the improvement in the testing R-squared after outlier removal is a positive sign. This suggests that removing outliers helped the model generalize slightly better, even if the primary issue of overfitting remains a consideration for further hyperparameter tuning.


**Reasoning**:
To retrain the Gradient Boosting Regressor model on the outlier-adjusted data, I will import the necessary class, instantiate the model, fit it to the cleaned training data, and then make predictions on both the cleaned training and testing sets.



In [18]:
from sklearn.ensemble import GradientBoostingRegressor

# Instantiate Gradient Boosting Regressor model with the same random_state
gbr_model_cleaned = GradientBoostingRegressor(random_state=42)

# Fit the model to the cleaned training data
gbr_model_cleaned.fit(X_train_cleaned, y_train_cleaned)

# Make predictions on cleaned training and testing sets
y_pred_train_gbr_cleaned = gbr_model_cleaned.predict(X_train_cleaned)
y_pred_test_gbr_cleaned = gbr_model_cleaned.predict(X_test_cleaned)

print("Gradient Boosting Regressor (Outlier-Adjusted) model trained and predictions made.")

Gradient Boosting Regressor (Outlier-Adjusted) model trained and predictions made.


**Reasoning**:
Now that the Gradient Boosting Regressor model has been retrained on the outlier-adjusted data and predictions have been made, I will calculate and print the R-squared scores for both the cleaned training and testing sets to evaluate its performance, as instructed by the overarching task structure.



In [19]:
from sklearn.metrics import r2_score

# Calculate R-squared for cleaned training set
r2_train_gbr_cleaned = r2_score(y_train_cleaned, y_pred_train_gbr_cleaned)
print(f"Gradient Boosting Regressor (Outlier-Adjusted) Training R-squared: {r2_train_gbr_cleaned:.4f}")

# Calculate R-squared for cleaned testing set
r2_test_gbr_cleaned = r2_score(y_test_cleaned, y_pred_test_gbr_cleaned)
print(f"Gradient Boosting Regressor (Outlier-Adjusted) Testing R-squared: {r2_test_gbr_cleaned:.4f}")

Gradient Boosting Regressor (Outlier-Adjusted) Training R-squared: 0.7213
Gradient Boosting Regressor (Outlier-Adjusted) Testing R-squared: 0.6634


The Gradient Boosting Regressor model, after outlier removal, shows a training R-squared of 0.7213 and a testing R-squared of 0.6634.

Compared to its performance before outlier removal (Training R-squared: 0.7349, Testing R-squared: 0.6662):
*   **Training R-squared** slightly decreased (from 0.7349 to 0.7213).
*   **Testing R-squared** slightly decreased (from 0.6662 to 0.6634).

The difference between training and testing R-squared remains small, indicating good generalization and less overfitting than the Random Forest model. However, unlike the Random Forest, outlier removal did not lead to an improvement in the testing R-squared for the Gradient Boosting Regressor; it remained largely similar or slightly worsened.

## Compare Models and Select the Best (Outlier-Adjusted)

### Subtask:
Compare the R-squared scores of all three re-trained models on both training and testing sets. Select the best performing model based on its generalization capabilities (testing R-squared score) while aiming for reduced overfitting.


### Model Comparison and Selection (Outlier-Adjusted Data)

Let's summarize the R-squared scores for all models after outlier removal:

*   **Linear Regression (Outlier-Adjusted)**
    *   Training R-squared: 0.1110
    *   Testing R-squared: 0.0995
    *   *Analysis*: Still very low R-squared, indicating severe underfitting. Performance slightly worsened after outlier removal compared to the original model, suggesting that linear models might not capture the underlying patterns effectively, or the removed data points were crucial for its (albeit poor) fit.

*   **Random Forest Regressor (Outlier-Adjusted)**
    *   Training R-squared: 0.9565
    *   Testing R-squared: 0.7011
    *   *Analysis*: Maintained a high training R-squared, with a slight improvement in testing R-squared (from 0.6901 to 0.7011) compared to the model before outlier removal. The large difference between training and testing still suggests overfitting, but the generalization to unseen data improved.

*   **Gradient Boosting Regressor (Outlier-Adjusted)**
    *   Training R-squared: 0.7213
    *   Testing R-squared: 0.6634
    *   *Analysis*: Training and testing R-squared scores are slightly lower than their pre-outlier removal counterparts (Training: 0.7349, Testing: 0.6662). This model shows less overfitting than the Random Forest, but its testing performance is slightly less than the outlier-adjusted Random Forest.

**Conclusion:**

Comparing the testing R-squared scores:
*   Linear Regression: 0.0995
*   Random Forest Regressor: 0.7011
*   Gradient Boosting Regressor: 0.6634

Based on the highest testing R-squared score, the **Random Forest Regressor (Outlier-Adjusted)** model is still the best performing model with a testing R-squared of **0.7011**. Although it still exhibits a noticeable gap between its training and testing scores, indicating some overfitting, its ability to generalize to unseen data (as measured by the testing R-squared) improved slightly after outlier removal. The Gradient Boosting Regressor, while showing less overfitting, did not achieve a higher testing R-squared in this specific instance. Therefore, for maximal predictive performance on unseen data, the Random Forest Regressor on the outlier-adjusted dataset is selected.

## Deploy Best Model with Gradio (Outlier-Adjusted)

### Subtask:
Create or update the Gradio web interface to deploy the newly selected best model, allowing users to input new data and get predictions. Ensure the deployment accounts for the outlier handling and feature scaling used in the best model's training pipeline.


**Reasoning**:
I will update the Gradio deployment to use the best performing model after outlier adjustment, which is the Random Forest Regressor (`rf_model_cleaned`). I will define a new prediction function, `predict_clv_cleaned`, that incorporates the necessary preprocessing steps (one-hot encoding, reindexing to match `X`'s original columns, and scaling with the global `scaler`) and then creates the Gradio interface with appropriate inputs and outputs.



In [20]:
import gradio as gr
import pandas as pd

# Best performing model is Random Forest Regressor (rf_model_cleaned) based on testing R-squared after outlier removal
# (r2_test_rf_cleaned: 0.7011)

# Get unique values for dropdowns from the 'df' (which is the one after dropping irrelevant columns but before one-hot encoding)
# This 'categorical_unique_values' is already available from previous steps.

def predict_clv_cleaned(*args):
    # The order of args must match the order of feature names defined for the Gradio interface
    # numerical_features: 'Income', 'Monthly Premium Auto', 'Months Since Last Claim', 'Months Since Policy Inception', 'Number of Open Complaints', 'Number of Policies', 'Total Claim Amount'
    # categorical_features: 'State', 'Response', 'Coverage', 'Education', 'EmploymentStatus', 'Gender', 'Location Code', 'Marital Status', 'Policy Type', 'Policy', 'Renew Offer Type', 'Sales Channel', 'Vehicle Class', 'Vehicle Size'

    input_data = {}
    arg_idx = 0
    # Populate numerical features
    for feature in numerical_features:
        input_data[feature] = [args[arg_idx]]
        arg_idx += 1
    # Populate categorical features
    for feature in categorical_features:
        input_data[feature] = [args[arg_idx]]
        arg_idx += 1

    input_df = pd.DataFrame(input_data)

    # Apply one-hot encoding to categorical features
    input_categorical_encoded = pd.get_dummies(input_df[categorical_features], drop_first=True)

    # Prepare the input_df for scaling and prediction
    # Drop original categorical columns from the input_df
    input_df_processed = input_df.drop(columns=categorical_features)
    # Concatenate with encoded features
    input_df_processed = pd.concat([input_df_processed, input_categorical_encoded], axis=1)

    # Reindex to ensure all columns from training (X.columns, from the original preprocessed X) are present and in the correct order
    # Fill missing columns (for categories not present in the single input row) with 0
    # This ensures consistency with the feature set X that rf_model_cleaned was trained on.
    input_df_processed = input_df_processed.reindex(columns=X.columns, fill_value=0)

    # Scale numerical features using the pre-fitted scaler
    input_df_processed[numerical_features] = scaler.transform(input_df_processed[numerical_features])

    # Make prediction using the best model (rf_model_cleaned)
    prediction = rf_model_cleaned.predict(input_df_processed)[0] # [0] to get the single scalar value

    return f"Predicted Customer Lifetime Value: ${prediction:.2f}"

# Create Gradio interface inputs (using existing categorical_unique_values and numerical_features)
inputs = []
for feature in numerical_features:
    inputs.append(gr.Number(label=feature))

for feature in categorical_features:
    inputs.append(gr.Dropdown(choices=categorical_unique_values[feature], label=feature))

output = gr.Textbox(label="Predicted Customer Lifetime Value")

# Launch the Gradio interface
gr.Interface(
    fn=predict_clv_cleaned,
    inputs=inputs,
    outputs=output,
    title="Customer Lifetime Value Prediction (Outlier-Adjusted)",
    description="Enter customer details to predict their Customer Lifetime Value using the outlier-adjusted model."
).launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://168308559fe6f0d2ea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Final Task

### Subtask:
Summarize the entire process, including the data preprocessing steps, the performance of each model, the rationale for selecting the best model, and a demonstration of the deployed Gradio interface for predicting Customer Lifetime Value.


## Summary:

### Q&A
**What was the impact of outlier removal on model performance?**
Outlier removal had a mixed impact on model performance. For the Linear Regression model, both training and testing R-squared scores decreased (from 0.1714 to 0.1110 for training and 0.1551 to 0.0995 for testing), indicating a worsening of its already poor fit. For the Random Forest Regressor, the testing R-squared slightly improved from 0.6901 to 0.7011, while the training R-squared slightly decreased from 0.9596 to 0.9565, suggesting a marginal improvement in generalization despite a slight increase in overfitting. The Gradient Boosting Regressor saw a slight decrease in both training (from 0.7349 to 0.7213) and testing (from 0.6662 to 0.6634) R-squared scores, meaning outlier removal did not benefit this model.

### Data Analysis Key Findings
*   **Outlier Removal Impact**: A total of 2698 rows, approximately 29.5% of the original dataset (9134 rows), were identified as outliers using the Interquartile Range (IQR) method and subsequently removed, resulting in a cleaned dataset of 6436 rows.
*   **Linear Regression Performance**: After outlier removal, the Linear Regression model continued to exhibit severe underfitting, with a training R-squared of 0.1110 and a testing R-squared of 0.0995. This was a decrease from its pre-outlier removal performance (training R-squared: 0.1714, testing R-squared: 0.1551).
*   **Random Forest Regressor Performance**: The Random Forest Regressor demonstrated the highest predictive power after outlier removal, achieving a training R-squared of 0.9565 and a testing R-squared of 0.7011. Its testing R-squared improved slightly from 0.6901 before outlier removal, suggesting better generalization.
*   **Gradient Boosting Regressor Performance**: The Gradient Boosting Regressor showed robust performance with less overfitting than Random Forest, yielding a training R-squared of 0.7213 and a testing R-squared of 0.6634. However, these scores were marginally lower than its performance before outlier removal (training R-squared: 0.7349, testing R-squared: 0.6662).
*   **Best Model Selection**: The Random Forest Regressor (Outlier-Adjusted) was selected as the best performing model due to its highest testing R-squared score of 0.7011, indicating the best generalization capability on unseen data among the evaluated models.
*   **Gradio Deployment**: The selected Random Forest Regressor model was successfully deployed via a Gradio web interface, allowing users to input customer details and receive predictions for Customer Lifetime Value, with all necessary preprocessing steps handled internally.

### Insights or Next Steps
*   Further hyperparameter tuning for the Random Forest Regressor could potentially reduce the gap between training and testing R-squared, mitigating overfitting while maintaining or improving generalization performance.
*   Given the significant portion of data removed due to outliers (29.5%), alternative outlier handling strategies, such as capping (winsorization) instead of removal, could be explored to retain more data and potentially improve model robustness for models that were negatively affected by complete removal.


# Task
## Log Transform Target Variable

### Subtask:
Apply a log transformation to the 'Customer Lifetime Value' (`y_cleaned`) to handle its skewed distribution. Create `y_log_transformed`.

## Log Transform Target Variable

### Subtask:
Apply a log transformation to the 'Customer Lifetime Value' (`y_cleaned`) to handle its skewed distribution. Create `y_log_transformed`.


**Reasoning**:
To apply a log transformation to the 'Customer Lifetime Value' (`y_cleaned`) and store it in `y_log_transformed`, I will import the `numpy` library and then use `np.log1p()` on `y_cleaned`. Finally, I will print the first 5 rows of the transformed series to verify the transformation.



In [21]:
import numpy as np

# Apply log transformation to y_cleaned
y_log_transformed = np.log1p(y_cleaned)

# Print the first 5 rows of y_log_transformed to verify the transformation
print("First 5 rows of y_log_transformed:")
print(y_log_transformed.head())

First 5 rows of y_log_transformed:
0    7.924622
2    9.464085
3    8.942051
4    7.942608
5    9.018853
Name: Customer Lifetime Value, dtype: float64


## Re-split Data into Training and Testing Sets (Log-Transformed)

### Subtask:
Divide the outlier-adjusted features (`X_cleaned`) and the log-transformed target variable (`y_log_transformed`) into new training and testing sets to prepare for re-training the models.


**Reasoning**:
To divide the outlier-adjusted features and log-transformed target variable into training and testing sets, I will use `train_test_split` with the specified parameters and then print the shapes of the resulting datasets to confirm the split.



In [22]:
from sklearn.model_selection import train_test_split

# Split the cleaned data with log-transformed target into training and testing sets
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X_cleaned, y_log_transformed, test_size=0.2, random_state=42)

print("Shape of X_train_log:", X_train_log.shape)
print("Shape of X_test_log:", X_test_log.shape)
print("Shape of y_train_log:", y_train_log.shape)
print("Shape of y_test_log:", y_test_log.shape)

Shape of X_train_log: (5148, 50)
Shape of X_test_log: (1288, 50)
Shape of y_train_log: (5148,)
Shape of y_test_log: (1288,)


## Retrain and Evaluate Linear Regression Model (Log-Transformed)

### Subtask:
Retrain the Linear Regression model on the log-transformed training data and evaluate its performance using R-squared scores for both training and testing sets. Predictions will be inverse-transformed for evaluation.


**Reasoning**:
To retrain and evaluate the Linear Regression model on the log-transformed data as instructed, I will first import the necessary libraries, instantiate the model, fit it to the log-transformed training data, make predictions, inverse transform these predictions, and then calculate and print the R-squared scores for both training and testing sets using the original scale of the target variable.



In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np

# Instantiate Linear Regression model
linear_model_log = LinearRegression()

# Fit the model to the log-transformed training data
linear_model_log.fit(X_train_log, y_train_log)

# Make predictions on log-transformed training and testing sets
y_pred_train_linear_log = linear_model_log.predict(X_train_log)
y_pred_test_linear_log = linear_model_log.predict(X_test_log)

# Inverse transform the predictions back to the original scale
y_pred_train_linear_orig = np.expm1(y_pred_train_linear_log)
y_pred_test_linear_orig = np.expm1(y_pred_test_linear_log)

# Calculate R-squared for training set (comparing with original cleaned target)
r2_train_linear_log_transformed = r2_score(y_train_cleaned.loc[y_train_log.index], y_pred_train_linear_orig)
print(f"Linear Regression (Log-Transformed) Training R-squared: {r2_train_linear_log_transformed:.4f}")

# Calculate R-squared for testing set (comparing with original cleaned target)
r2_test_linear_log_transformed = r2_score(y_test_cleaned.loc[y_test_log.index], y_pred_test_linear_orig)
print(f"Linear Regression (Log-Transformed) Testing R-squared: {r2_test_linear_log_transformed:.4f}")

Linear Regression (Log-Transformed) Training R-squared: 0.0408
Linear Regression (Log-Transformed) Testing R-squared: 0.0248


The Linear Regression model, after log transformation and outlier removal, shows a training R-squared of 0.0408 and a testing R-squared of 0.0248. These scores are extremely low, indicating that the model performs very poorly. Comparing these to the previous Linear Regression models (before outlier removal: Training R-squared: 0.1714, Testing R-squared: 0.1551; after outlier removal: Training R-squared: 0.1110, Testing R-squared: 0.0995), the log transformation appears to have further degraded the Linear Regression model's performance. This suggests severe underfitting, as the model is unable to capture any meaningful patterns in the data even after these transformations. The very small difference between training and testing R-squared indicates no overfitting, but rather a consistent lack of fit across both sets.

**Reasoning**:
Now, following the pattern of the previous subtasks, I will retrain the Random Forest Regressor model on the log-transformed data, instantiate the model, fit it to the `X_train_log` and `y_train_log` data, and then make predictions on both the training and testing sets.



In [24]:
from sklearn.ensemble import RandomForestRegressor

# Instantiate Random Forest Regressor model with the same random_state
rf_model_log = RandomForestRegressor(random_state=42)

# Fit the model to the log-transformed training data
rf_model_log.fit(X_train_log, y_train_log)

# Make predictions on log-transformed training and testing sets
y_pred_train_rf_log = rf_model_log.predict(X_train_log)
y_pred_test_rf_log = rf_model_log.predict(X_test_log)

print("Random Forest Regressor (Log-Transformed) model trained and predictions made.")

Random Forest Regressor (Log-Transformed) model trained and predictions made.


**Reasoning**:
Now that the Random Forest Regressor model has been retrained on the log-transformed data and predictions have been made, I will calculate and print the R-squared scores for both the training and testing sets, remembering to inverse transform the predictions to compare against the original scaled target variable.



In [25]:
from sklearn.metrics import r2_score
import numpy as np

# Inverse transform the predictions back to the original scale
y_pred_train_rf_orig = np.expm1(y_pred_train_rf_log)
y_pred_test_rf_orig = np.expm1(y_pred_test_rf_log)

# Calculate R-squared for training set (comparing with original cleaned target)
r2_train_rf_log_transformed = r2_score(y_train_cleaned.loc[y_train_log.index], y_pred_train_rf_orig)
print(f"Random Forest Regressor (Log-Transformed) Training R-squared: {r2_train_rf_log_transformed:.4f}")

# Calculate R-squared for testing set (comparing with original cleaned target)
r2_test_rf_log_transformed = r2_score(y_test_cleaned.loc[y_test_log.index], y_pred_test_rf_orig)
print(f"Random Forest Regressor (Log-Transformed) Testing R-squared: {r2_test_rf_log_transformed:.4f}")

Random Forest Regressor (Log-Transformed) Training R-squared: 0.9426
Random Forest Regressor (Log-Transformed) Testing R-squared: 0.6947


The Random Forest Regressor model, after log transformation and outlier removal, shows a training R-squared of 0.9426 and a testing R-squared of 0.6947.

Compared to its performance before log transformation (but after outlier removal) (Training R-squared: 0.9565, Testing R-squared: 0.7011):
*   **Training R-squared** slightly decreased (from 0.9565 to 0.9426).
*   **Testing R-squared** also slightly decreased (from 0.7011 to 0.6947).

While the model still shows a high training R-squared and a notable difference between its training and testing R-squared, indicating some level of overfitting, the log transformation did not improve the testing R-squared. In this case, removing outliers improved the RFR model slightly, but applying a log transformation to the target variable did not lead to further improvements in its generalization capability as measured by the testing R-squared.

**Reasoning**:
Now, I will retrain the Gradient Boosting Regressor model on the log-transformed data, instantiate the model, fit it to the `X_train_log` and `y_train_log` data, and then make predictions on both the training and testing sets.



In [26]:
from sklearn.ensemble import GradientBoostingRegressor

# Instantiate Gradient Boosting Regressor model with the same random_state
gbr_model_log = GradientBoostingRegressor(random_state=42)

# Fit the model to the log-transformed training data
gbr_model_log.fit(X_train_log, y_train_log)

# Make predictions on log-transformed training and testing sets
y_pred_train_gbr_log = gbr_model_log.predict(X_train_log)
y_pred_test_gbr_log = gbr_model_log.predict(X_test_log)

print("Gradient Boosting Regressor (Log-Transformed) model trained and predictions made.")

Gradient Boosting Regressor (Log-Transformed) model trained and predictions made.


**Reasoning**:
Now that the Gradient Boosting Regressor model has been retrained on the log-transformed data and predictions have been made, I will calculate and print the R-squared scores for both the training and testing sets, remembering to inverse transform the predictions to compare against the original scaled target variable.



In [27]:
from sklearn.metrics import r2_score
import numpy as np

# Inverse transform the predictions back to the original scale
y_pred_train_gbr_orig = np.expm1(y_pred_train_gbr_log)
y_pred_test_gbr_orig = np.expm1(y_pred_test_gbr_log)

# Calculate R-squared for training set (comparing with original cleaned target)
r2_train_gbr_log_transformed = r2_score(y_train_cleaned.loc[y_train_log.index], y_pred_train_gbr_orig)
print(f"Gradient Boosting Regressor (Log-Transformed) Training R-squared: {r2_train_gbr_log_transformed:.4f}")

# Calculate R-squared for testing set (comparing with original cleaned target)
r2_test_gbr_log_transformed = r2_score(y_test_cleaned.loc[y_test_log.index], y_pred_test_gbr_orig)
print(f"Gradient Boosting Regressor (Log-Transformed) Testing R-squared: {r2_test_gbr_log_transformed:.4f}")

Gradient Boosting Regressor (Log-Transformed) Training R-squared: 0.6963
Gradient Boosting Regressor (Log-Transformed) Testing R-squared: 0.6523


The Gradient Boosting Regressor model, after log transformation and outlier removal, shows a training R-squared of 0.6963 and a testing R-squared of 0.6523.

Compared to its performance before log transformation (but after outlier removal) (Training R-squared: 0.7213, Testing R-squared: 0.6634):
*   **Training R-squared** decreased (from 0.7213 to 0.6963).
*   **Testing R-squared** also decreased (from 0.6634 to 0.6523).

The log transformation had a slightly negative impact on the Gradient Boosting Regressor's performance, resulting in marginally lower R-squared scores for both training and testing sets. While the difference between training and testing R-squared remains relatively small, suggesting good generalization and less overfitting compared to the Random Forest model, the log transformation did not yield an improvement for this model in this scenario.

## Compare Models and Select the Best (Log-Transformed and Outlier-Adjusted)

### Subtask:
Compare the R-squared scores of all three re-trained models on both training and testing sets. Select the best performing model based on its generalization capabilities (testing R-squared score) while aiming for reduced overfitting.

### Model Comparison and Selection (Log-Transformed and Outlier-Adjusted Data)

Let's summarize the R-squared scores for all models after log transformation and outlier removal:

*   **Linear Regression (Log-Transformed and Outlier-Adjusted)**
    *   Training R-squared: 0.0408
    *   Testing R-squared: 0.0248
    *   *Analysis*: Performance significantly worsened compared to previous iterations, indicating severe underfitting. The log transformation, in combination with outlier removal, proved detrimental for this model, suggesting it's unsuitable for this dataset and transformations.

*   **Random Forest Regressor (Log-Transformed and Outlier-Adjusted)**
    *   Training R-squared: 0.9426
    *   Testing R-squared: 0.6947
    *   *Analysis*: Maintains a high training R-squared, but the testing R-squared (0.6947) is slightly lower than the Random Forest model with only outlier removal (0.7011). This suggests that log transformation did not further improve its generalization and might have slightly hindered it. Overfitting is still a concern due to the large gap between training and testing scores.

*   **Gradient Boosting Regressor (Log-Transformed and Outlier-Adjusted)**
    *   Training R-squared: 0.6963
    *   Testing R-squared: 0.6523
    *   *Analysis*: Both training and testing R-squared scores decreased compared to the Gradient Boosting model with only outlier removal (Training: 0.7213, Testing: 0.6634). The log transformation had a slightly negative impact. It still shows less overfitting than Random Forest, but its predictive power on the test set is now lower than the Random Forest model.

**Overall Conclusion After Outlier Removal and Log Transformation:**

Comparing the testing R-squared scores:
*   Linear Regression: 0.0248
*   Random Forest Regressor: 0.6947
*   Gradient Boosting Regressor: 0.6523

The **Random Forest Regressor (Outlier-Adjusted)** without log transformation remains the best performing model so far, achieving a testing R-squared of **0.7011**. The Random Forest Regressor (Log-Transformed and Outlier-Adjusted) is a close second with 0.6947. Given that the log transformation slightly decreased the performance of both Random Forest and Gradient Boosting models, and severely worsened Linear Regression, it seems that for this dataset, the log transformation did not provide the expected benefits. Therefore, the **Random Forest Regressor (Outlier-Adjusted)** model (from the previous subtask) is still considered the best model for deployment, as it demonstrated the highest testing R-squared.

## Summary:

### Q&A
**What was the impact of outlier removal on model performance?**
Outlier removal had a mixed impact on model performance. For the Linear Regression model, both training and testing R-squared scores decreased (from 0.1714 to 0.1110 for training and 0.1551 to 0.0995 for testing), indicating a worsening of its already poor fit. For the Random Forest Regressor, the testing R-squared slightly improved from 0.6901 to 0.7011, while the training R-squared slightly decreased from 0.9596 to 0.9565, suggesting a marginal improvement in generalization despite a slight increase in overfitting. The Gradient Boosting Regressor saw a slight decrease in both training (from 0.7349 to 0.7213) and testing (from 0.6662 to 0.6634) R-squared scores, meaning outlier removal did not benefit this model.

**What was the impact of log transformation on model performance?**
Applying a log transformation to the target variable after outlier removal generally did not improve model performance for this dataset. For the Linear Regression model, log transformation further degraded performance (testing R-squared dropped from 0.0995 to 0.0248), indicating severe underfitting. For the Random Forest Regressor, log transformation resulted in a slight decrease in testing R-squared (from 0.7011 to 0.6947). Similarly, for the Gradient Boosting Regressor, testing R-squared slightly decreased (from 0.6634 to 0.6523). This suggests that for these models and this dataset, the raw or simply outlier-adjusted target variable was more suitable than its log-transformed version, as log transformation did not lead to improved predictive power or reduced overfitting in this case.

### Data Analysis Key Findings
*   **Outlier Removal Impact**: A total of 2698 rows, approximately 29.5% of the original dataset (9134 rows), were identified as outliers using the Interquartile Range (IQR) method and subsequently removed, resulting in a cleaned dataset of 6436 rows.
*   **Linear Regression Performance**:
    *   Before outlier/log transform: Training R-squared: 0.1714, Testing R-squared: 0.1551.
    *   After outlier removal: Training R-squared: 0.1110, Testing R-squared: 0.0995.
    *   After outlier and log transform: Training R-squared: 0.0408, Testing R-squared: 0.0248. The model consistently suffered from severe underfitting, with performance degrading further after transformations.
*   **Random Forest Regressor Performance**:
    *   Before outlier/log transform: Training R-squared: 0.9596, Testing R-squared: 0.6901.
    *   After outlier removal: Training R-squared: 0.9565, Testing R-squared: 0.7011. This was the best performance overall.
    *   After outlier and log transform: Training R-squared: 0.9426, Testing R-squared: 0.6947. Performance slightly declined after log transformation. The model consistently showed signs of overfitting (large gap between training and testing R-squared).
*   **Gradient Boosting Regressor Performance**:
    *   Before outlier/log transform: Training R-squared: 0.7349, Testing R-squared: 0.6662.
    *   After outlier removal: Training R-squared: 0.7213, Testing R-squared: 0.6634.
    *   After outlier and log transform: Training R-squared: 0.6963, Testing R-squared: 0.6523. Performance slightly declined after both transformations. This model generally showed less overfitting than Random Forest.
*   **Best Model Selection**: The **Random Forest Regressor (Outlier-Adjusted)**, trained on data with outliers removed but *without* log transformation, was selected as the best performing model due to its highest testing R-squared score of **0.7011**. This model demonstrated the best generalization capability on unseen data among all evaluated scenarios.
*   **Gradio Deployment**: The selected Random Forest Regressor model (trained on outlier-adjusted data) was successfully deployed via a Gradio web interface, allowing users to input customer details and receive predictions for Customer Lifetime Value, with all necessary preprocessing steps handled internally.

### Insights or Next Steps
*   **Overfitting Mitigation**: While the Random Forest Regressor had the best testing performance, the notable difference between its training and testing R-squared scores indicates overfitting. Further hyperparameter tuning (e.g., using GridSearchCV or RandomizedSearchCV) or techniques like pruning could be explored to improve its generalization capabilities and potentially reduce this gap without sacrificing too much predictive power.
*   **Alternative Outlier Handling**: Given the significant portion of data removed due to outliers (29.5%), alternative outlier handling strategies, such as capping (winsorization) instead of complete removal, could be explored to retain more data. This might be especially beneficial for models that saw a performance decrease after outlier removal (like Linear Regression and Gradient Boosting).
*   **Advanced Feature Engineering**: Given the overall R-squared values, exploring more advanced feature engineering, such as interaction terms, polynomial features, or different ways to encode categorical variables, could enhance the models' ability to capture more complex relationships and further improve predictive accuracy.
*   **Different Model Architectures**: Experimenting with other regression models or ensemble techniques could potentially yield better results.

## Summary:

### Q&A
**What was the impact of outlier removal on model performance?**
Outlier removal had a mixed impact on model performance. For the Linear Regression model, both training and testing R-squared scores decreased (from 0.1714 to 0.1110 for training and 0.1551 to 0.0995 for testing), indicating a worsening of its already poor fit. For the Random Forest Regressor, the testing R-squared slightly improved from 0.6901 to 0.7011, while the training R-squared slightly decreased from 0.9596 to 0.9565, suggesting a marginal improvement in generalization despite a slight increase in overfitting. The Gradient Boosting Regressor saw a slight decrease in both training (from 0.7349 to 0.7213) and testing (from 0.6662 to 0.6634) R-squared scores, meaning outlier removal did not benefit this model.

**What was the impact of log transformation on model performance?**
Applying a log transformation to the target variable after outlier removal generally did not improve model performance for this dataset. For the Linear Regression model, log transformation further degraded performance (testing R-squared dropped from 0.0995 to 0.0248), indicating severe underfitting. For the Random Forest Regressor, log transformation resulted in a slight decrease in testing R-squared (from 0.7011 to 0.6947). Similarly, for the Gradient Boosting Regressor, testing R-squared slightly decreased (from 0.6634 to 0.6523). This suggests that for these models and this dataset, the raw or simply outlier-adjusted target variable was more suitable than its log-transformed version, as log transformation did not lead to improved predictive power or reduced overfitting in this case.

### Data Analysis Key Findings
*   **Outlier Removal Impact**: A total of 2698 rows, approximately 29.5% of the original dataset (9134 rows), were identified as outliers using the Interquartile Range (IQR) method and subsequently removed, resulting in a cleaned dataset of 6436 rows.
*   **Linear Regression Performance**:
    *   Before outlier/log transform: Training R-squared: 0.1714, Testing R-squared: 0.1551.
    *   After outlier removal: Training R-squared: 0.1110, Testing R-squared: 0.0995.
    *   After outlier and log transform: Training R-squared: 0.0408, Testing R-squared: 0.0248. The model consistently suffered from severe underfitting, with performance degrading further after transformations.
*   **Random Forest Regressor Performance**:
    *   Before outlier/log transform: Training R-squared: 0.9596, Testing R-squared: 0.6901.
    *   After outlier removal: Training R-squared: 0.9565, Testing R-squared: 0.7011. This was the best performance overall.
    *   After outlier and log transform: Training R-squared: 0.9426, Testing R-squared: 0.6947. Performance slightly declined after log transformation. The model consistently showed signs of overfitting (large gap between training and testing R-squared).
*   **Gradient Boosting Regressor Performance**:
    *   Before outlier/log transform: Training R-squared: 0.7349, Testing R-squared: 0.6662.
    *   After outlier removal: Training R-squared: 0.7213, Testing R-squared: 0.6634.
    *   After outlier and log transform: Training R-squared: 0.6963, Testing R-squared: 0.6523. Performance slightly declined after both transformations. This model generally showed less overfitting than Random Forest.
*   **Best Model Selection**: The **Random Forest Regressor (Outlier-Adjusted)**, trained on data with outliers removed but *without* log transformation, was selected as the best performing model due to its highest testing R-squared score of **0.7011**. This model demonstrated the best generalization capability on unseen data among all evaluated scenarios.
*   **Gradio Deployment**: The selected Random Forest Regressor model (trained on outlier-adjusted data) was successfully deployed via a Gradio web interface, allowing users to input customer details and receive predictions for Customer Lifetime Value, with all necessary preprocessing steps handled internally.

### Insights or Next Steps
*   **Overfitting Mitigation**: While the Random Forest Regressor had the best testing performance, the notable difference between its training and testing R-squared scores indicates overfitting. Further hyperparameter tuning (e.g., using GridSearchCV or RandomizedSearchCV) or techniques like pruning could be explored to improve its generalization capabilities and potentially reduce this gap without sacrificing too much predictive power.
*   **Alternative Outlier Handling**: Given the significant portion of data removed due to outliers (29.5%), alternative outlier handling strategies, such as capping (winsorization) instead of complete removal, could be explored to retain more data. This might be especially beneficial for models that saw a performance decrease after outlier removal (like Linear Regression and Gradient Boosting).
*   **Advanced Feature Engineering**: Given the overall R-squared values, exploring more advanced feature engineering, such as interaction terms, polynomial features, or different ways to encode categorical variables, could enhance the models' ability to capture more complex relationships and further improve predictive accuracy.
*   **Different Model Architectures**: Experimenting with other regression models or ensemble techniques could potentially yield better results.


## Summary:

### Q&A
**What was the impact of outlier removal on model performance?**
Outlier removal had a mixed impact on model performance. For the Linear Regression model, both training and testing R-squared scores decreased (from 0.1714 to 0.1110 for training and 0.1551 to 0.0995 for testing), indicating a worsening of its already poor fit. For the Random Forest Regressor, the testing R-squared slightly improved from 0.6901 to 0.7011, while the training R-squared slightly decreased from 0.9596 to 0.9565, suggesting a marginal improvement in generalization despite a slight increase in overfitting. The Gradient Boosting Regressor saw a slight decrease in both training (from 0.7349 to 0.7213) and testing (from 0.6662 to 0.6634) R-squared scores, meaning outlier removal did not benefit this model.

**What was the impact of log transformation on model performance?**
Applying a log transformation to the target variable after outlier removal generally did not improve model performance for this dataset. For the Linear Regression model, log transformation further degraded performance (testing R-squared dropped from 0.0995 to 0.0248), indicating severe underfitting. For the Random Forest Regressor, log transformation resulted in a slight decrease in testing R-squared (from 0.7011 to 0.6947). Similarly, for the Gradient Boosting Regressor, testing R-squared slightly decreased (from 0.6634 to 0.6523). This suggests that for these models and this dataset, the raw or simply outlier-adjusted target variable was more suitable than its log-transformed version, as log transformation did not lead to improved predictive power or reduced overfitting in this case.

### Data Analysis Key Findings
*   **Outlier Removal Impact**: A total of 2698 rows, approximately 29.5% of the original dataset (9134 rows), were identified as outliers using the Interquartile Range (IQR) method and subsequently removed, resulting in a cleaned dataset of 6436 rows.
*   **Linear Regression Performance**:
    *   Before outlier/log transform: Training R-squared: 0.1714, Testing R-squared: 0.1551.
    *   After outlier removal: Training R-squared: 0.1110, Testing R-squared: 0.0995.
    *   After outlier and log transform: Training R-squared: 0.0408, Testing R-squared: 0.0248. The model consistently suffered from severe underfitting, with performance degrading further after transformations.
*   **Random Forest Regressor Performance**:
    *   Before outlier/log transform: Training R-squared: 0.9596, Testing R-squared: 0.6901.
    *   After outlier removal: Training R-squared: 0.9565, Testing R-squared: 0.7011. This was the best performance overall.
    *   After outlier and log transform: Training R-squared: 0.9426, Testing R-squared: 0.6947. Performance slightly declined after log transformation. The model consistently showed signs of overfitting (large gap between training and testing R-squared).
*   **Gradient Boosting Regressor Performance**:
    *   Before outlier/log transform: Training R-squared: 0.7349, Testing R-squared: 0.6662.
    *   After outlier removal: Training R-squared: 0.7213, Testing R-squared: 0.6634.
    *   After outlier and log transform: Training R-squared: 0.6963, Testing R-squared: 0.6523. Performance slightly declined after both transformations. This model generally showed less overfitting than Random Forest.
*   **Best Model Selection**: The **Random Forest Regressor (Outlier-Adjusted)**, trained on data with outliers removed but *without* log transformation, was selected as the best performing model due to its highest testing R-squared score of **0.7011**. This model demonstrated the best generalization capability on unseen data among all evaluated scenarios.
*   **Gradio Deployment**: The selected Random Forest Regressor model (trained on outlier-adjusted data) was successfully deployed via a Gradio web interface, allowing users to input customer details and receive predictions for Customer Lifetime Value, with all necessary preprocessing steps handled internally.

### Insights or Next Steps
*   **Overfitting Mitigation**: While the Random Forest Regressor had the best testing performance, the notable difference between its training and testing R-squared scores indicates overfitting. Further hyperparameter tuning (e.g., using GridSearchCV or RandomizedSearchCV) or techniques like pruning could be explored to improve its generalization capabilities and potentially reduce this gap without sacrificing too much predictive power.
*   **Alternative Outlier Handling**: Given the significant portion of data removed due to outliers (29.5%), alternative outlier handling strategies, such as capping (winsorization) instead of complete removal, could be explored to retain more data. This might be especially beneficial for models that saw a performance decrease after outlier removal (like Linear Regression and Gradient Boosting).
*   **Advanced Feature Engineering**: Given the overall R-squared values, exploring more advanced feature engineering, such as interaction terms, polynomial features, or different ways to encode categorical variables, could enhance the models' ability to capture more complex relationships and further improve predictive accuracy.
*   **Different Model Architectures**: Experimenting with other regression models or ensemble techniques could potentially yield better results.

## Compare Models and Select the Best (Log-Transformed)

### Subtask:
Compare the R-squared scores of all three re-trained models on both training and testing sets with log-transformed target. Select the best performing model based on its generalization capabilities (testing R-squared score) while aiming for reduced overfitting. Ensure that the R-squared is calculated on inverse-transformed predictions against the original y_test values.


### Model Comparison and Selection (Log-Transformed Target)

Let's summarize the R-squared scores for all models after log transformation of the target variable and outlier removal:

*   **Linear Regression (Log-Transformed)**
    *   Training R-squared: 0.0408
    *   Testing R-squared: 0.0248
    *   *Analysis*: The performance of the Linear Regression model significantly degraded after log transformation, resulting in extremely low R-squared scores. This model continues to severely underfit the data.

*   **Random Forest Regressor (Log-Transformed)**
    *   Training R-squared: 0.9426
    *   Testing R-squared: 0.6947
    *   *Analysis*: This model maintains a high training R-squared, but the testing R-squared is slightly lower than its performance on the outlier-adjusted data without log transformation (0.7011). The gap between training and testing scores indicates continued overfitting, though its testing performance is still competitive.

*   **Gradient Boosting Regressor (Log-Transformed)**
    *   Training R-squared: 0.6963
    *   Testing R-squared: 0.6523
    *   *Analysis*: Both training and testing R-squared scores for this model are slightly lower compared to its performance on the outlier-adjusted data without log transformation (Training: 0.7213, Testing: 0.6634). It shows less overfitting than the Random Forest, but its testing performance is the lowest among the tree-based models after log transformation.

**Conclusion:**

Comparing the testing R-squared scores:
*   Linear Regression: 0.0248
*   Random Forest Regressor: 0.6947
*   Gradient Boosting Regressor: 0.6523

Based on the highest testing R-squared score, the **Random Forest Regressor (Log-Transformed)** model still performs the best with a testing R-squared of **0.6947**. Although the log transformation slightly decreased its testing R-squared compared to the outlier-adjusted model without log transformation (0.7011), it still outperforms the other models significantly. It exhibits some overfitting (difference between training and testing R-squared is 0.2479), but its predictive power on unseen data is superior. Therefore, the Random Forest Regressor trained on the outlier-adjusted data with a log-transformed target variable is selected as the best model for this iteration, considering its overall highest predictive performance.

## Deploy Best Model with Gradio (Log-Transformed)

### Subtask:
Create or update the Gradio web interface to deploy the newly selected best model, allowing users to input new data and get predictions. Ensure the deployment accounts for the log transformation and inverse log transformation for predictions, as well as prior preprocessing steps.


**Reasoning**:
I will define the `predict_clv_log_transformed` function to handle new data inputs, preprocess them (one-hot encode categorical features, reindex to match training columns, scale numerical features, and inverse transform the prediction), and then create and launch a Gradio interface with appropriate input and output components to deploy the best performing model (`rf_model_log`).



In [28]:
import gradio as gr
import pandas as pd
import numpy as np

# Best performing model is Random Forest Regressor (rf_model_log) based on testing R-squared after log transformation
# (r2_test_rf_log_transformed: 0.6947)

def predict_clv_log_transformed(*args):
    # The order of args must match the order of feature names defined for the Gradio interface
    # numerical_features: 'Income', 'Monthly Premium Auto', 'Months Since Last Claim', 'Months Since Policy Inception', 'Number of Open Complaints', 'Number of Policies', 'Total Claim Amount'
    # categorical_features: 'State', 'Response', 'Coverage', 'Education', 'EmploymentStatus', 'Gender', 'Location Code', 'Marital Status', 'Policy Type', 'Policy', 'Renew Offer Type', 'Sales Channel', 'Vehicle Class', 'Vehicle Size'

    input_data = {}
    arg_idx = 0
    # Populate numerical features
    for feature in numerical_features:
        input_data[feature] = [args[arg_idx]]
        arg_idx += 1
    # Populate categorical features
    for feature in categorical_features:
        input_data[feature] = [args[arg_idx]]
        arg_idx += 1

    input_df = pd.DataFrame(input_data)

    # Apply one-hot encoding to categorical features
    input_categorical_encoded = pd.get_dummies(input_df[categorical_features], drop_first=True)

    # Prepare the input_df for scaling and prediction
    # Drop original categorical columns from the input_df
    input_df_processed = input_df.drop(columns=categorical_features)
    # Concatenate with encoded features
    input_df_processed = pd.concat([input_df_processed, input_categorical_encoded], axis=1)

    # Reindex to ensure all columns from training (X.columns, from the original preprocessed X) are present and in the correct order
    # Fill missing columns (for categories not present in the single input row) with 0
    # X.columns represents the feature set after one-hot encoding before outlier removal and log transform, which is consistent with X_cleaned and X_train_log column structure.
    input_df_processed = input_df_processed.reindex(columns=X.columns, fill_value=0)

    # Scale numerical features using the pre-fitted scaler
    input_df_processed[numerical_features] = scaler.transform(input_df_processed[numerical_features])

    # Make prediction using the best model (rf_model_log)
    predicted_log_clv = rf_model_log.predict(input_df_processed)[0]

    # Inverse transform the prediction back to the original scale
    prediction = np.expm1(predicted_log_clv)

    return f"Predicted Customer Lifetime Value: ${prediction:.2f}"

# Create Gradio interface inputs (using existing categorical_unique_values and numerical_features)
inputs = []
for feature in numerical_features:
    inputs.append(gr.Number(label=feature))

for feature in categorical_features:
    inputs.append(gr.Dropdown(choices=categorical_unique_values[feature], label=feature))

output = gr.Textbox(label="Predicted Customer Lifetime Value")

# Launch the Gradio interface
gr.Interface(
    fn=predict_clv_log_transformed,
    inputs=inputs,
    outputs=output,
    title="Customer Lifetime Value Prediction (Outlier-Adjusted, Log-Transformed)",
    description="Enter customer details to predict their Customer Lifetime Value using the outlier-adjusted and log-transformed model."
).launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0dfd4fd360bd606027.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Final Task

### Subtask:
Summarize the entire process, focusing on the impact of log transformation on model performance, highlight the final best model, and demonstrate the updated Gradio interface.


## Summary:

### Q&A
**What was the impact of log transformation on model performance?**
Applying a log transformation to the target variable (`Customer Lifetime Value`) after outlier removal generally did not improve model performance for this dataset.
*   For the Linear Regression model, log transformation further degraded performance, with the testing R-squared dropping significantly from 0.0995 to 0.0248, indicating severe underfitting.
*   For the Random Forest Regressor, log transformation resulted in a slight decrease in testing R-squared from 0.7011 to 0.6947.
*   Similarly, for the Gradient Boosting Regressor, testing R-squared slightly decreased from 0.6634 to 0.6523.
This suggests that for these models and this dataset, the raw or simply outlier-adjusted target variable was more suitable than its log-transformed version, as log transformation did not lead to improved predictive power or reduced overfitting.

### Data Analysis Key Findings
*   **Log Transformation Application**: The 'Customer Lifetime Value' (`y_cleaned`) was successfully log-transformed using `np.log1p()` to create `y_log_transformed` to address its skewed distribution.
*   **Data Resplitting**: The outlier-adjusted features (`X_cleaned`) and the new log-transformed target (`y_log_transformed`) were re-split into training and testing sets, resulting in `X_train_log` (5148, 50), `X_test_log` (1288, 50), `y_train_log` (5148,), and `y_test_log` (1288,).
*   **Linear Regression (Log-Transformed) Performance**: The Linear Regression model, after log transformation and outlier removal, showed extremely poor performance with a training R-squared of 0.0408 and a testing R-squared of 0.0248, indicating severe underfitting and a degradation from previous iterations.
*   **Random Forest Regressor (Log-Transformed) Performance**: This model achieved a training R-squared of 0.9426 and a testing R-squared of 0.6947. While still the best among the log-transformed models, its testing R-squared slightly decreased from the Random Forest model with only outlier removal (0.7011).
*   **Gradient Boosting Regressor (Log-Transformed) Performance**: This model yielded a training R-squared of 0.6963 and a testing R-squared of 0.6523. Similar to Random Forest, its performance slightly declined after log transformation compared to the outlier-adjusted model without it (testing R-squared of 0.6634).
*   **Best Model Selection**: Despite a slight reduction in testing R-squared compared to the Random Forest model without log transformation (0.7011 vs. 0.6947), the **Random Forest Regressor trained on the outlier-adjusted data with a log-transformed target variable** was selected as the best model for this iteration, demonstrating the highest predictive performance (testing R-squared of 0.6947) among the log-transformed models.
*   **Gradio Deployment**: The selected Random Forest Regressor model (`rf_model_log`) was successfully deployed via a Gradio web interface, which is accessible via a public URL `https://0dfd4fd360bd606027.gradio.live`. The interface properly handles all preprocessing steps (one-hot encoding, reindexing, numerical scaling) and performs inverse log transformation on predictions to present Customer Lifetime Value in its original scale.

### Insights or Next Steps
*   **Re-evaluate Log Transformation**: Given that log transformation did not universally improve model performance and even degraded some, further investigation into the optimal transformation for the target variable, or even refraining from transformation, should be considered for this dataset.
*   **Hyperparameter Tuning for Overfitting**: The chosen Random Forest Regressor still exhibits a notable gap between its training (0.9426) and testing (0.6947) R-squared scores, indicating overfitting. Advanced hyperparameter tuning techniques (e.g., GridSearchCV, RandomizedSearchCV) could be employed to optimize model complexity and improve generalization.
